In [1]:
input_path = "input"
output_path = "output"

seed = 0

In [2]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import itertools
import geopandas as gpd
import os

/home/shoerl/anaconda3/envs/lead_lyon/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [3]:
assert os.path.exists("%s/area.gpkg" % input_path)
assert os.path.exists("%s/vrp_network.gpkg" % input_path)
assert os.path.exists("%s/vrp_nodes.gpkg" % input_path)

assert os.path.exists("%s/slots.csv" % input_path)
assert os.path.exists("%s/homes.gpkg" % input_path)

# Relevant information

In [4]:
df_spatial = gpd.read_file("%s/homes.gpkg" % input_path)

In [5]:
df_slots = pd.read_csv("%s/slots.csv" % input_path, sep = ";")

# Filter Confluence households

In [6]:
confluence_households = gpd.sjoin(
    df_spatial,
    gpd.read_file("%s/area.gpkg" % input_path),
    op = "within"
)["household_id"].unique()

In [7]:
df_confluence = df_slots[df_slots["household_id"].isin(confluence_households)]

In [8]:
df_nodes = gpd.read_file("%s/vrp_nodes.gpkg" % input_path)

In [9]:
# Find household nodes
import sklearn
import sklearn.neighbors

node_index = sklearn.neighbors.KDTree(
    np.vstack([df_nodes["geometry"].x.values, df_nodes["geometry"].y.values]).T
)

df_households = df_slots.drop_duplicates("household_id")[["household_id", "x", "y"]]

df_households["location_id"] = df_nodes.iloc[node_index.query(
    np.vstack([df_households["x"].values, df_households["y"].values]).T
)[1].flatten()]["id"].values

In [10]:
df_deliveries = pd.merge(df_confluence, df_households[["household_id", "location_id"]])

df_deliveries.to_csv(
    "%s/vrp_deliveries.csv" % output_path, sep = ";"
)